1. Описание задания
Исходные данные:
Тренер сборной команды по плаванию должен выбрать четырех пловцов из пяти для участия в эстафете на 200 м. Эстафета состоит из 4 этапов, каждый из которых плывется определенным стилем. В таблице приведены результаты (в секундах), которые пловцы показывают на 50-метровом этапе каждым стилем.
Этап (Стиль)	Пловец 1	Пловец 2	Пловец 3	Пловец 4	Пловец 5
1	37.7	32.9	33.8	37.0	35.4
2	43.4	33.1	42.2	34.7	41.8
3	33.3	28.5	38.9	30.4	33.6
4	29.2	26.4	29.6	28.5	31.1
Цель:
Минимизировать суммарное время прохождения эстафеты.
Задания для анализа:
Выбрать четырех пловцов из пяти и распределить их по этапам так, чтобы итоговое время было наименьшим. Какова величина этого результата?
Как изменится решение, если дополнительно потребовать, чтобы 5-й пловец обязательно участвовал в эстафете?

Ожидаемый результат (на выходе):
Оптимальный состав команды из 4-х человек.
Распределение пловцов по этапам.
Минимальное суммарное время эстафеты.
Новый план и новое время с учетом обязательного участия 5-го пловца, а также анализ изменений.
2. Математическая модель задачи
Это задача о назначениях.
Переменные решения:
Введем бинарную переменную x_ij, которая равна 1, если пловец i назначен на этап j, и 0 в противном случае.
i — индекс пловца (от 1 до 5)
j — индекс этапа (от 1 до 4)
Параметры:
t_ij — время (стоимость), которое пловец i тратит на прохождение этапа j.
Целевая функция:
Минимизировать суммарное время всех назначенных пловцов на их этапы.
Z = Σ (от i=1 до 5) Σ (от j=1 до 4) (t_ij * x_ij) -> min
Ограничения:
Каждый этап должен быть пройден ровно одним пловцом:
Σ (от i=1 до 5) x_ij = 1 для каждого этапа j (от 1 до 4).
Каждый пловец может быть назначен не более чем на один этап:
Σ (от j=1 до 4) x_ij ≤ 1 для каждого пловца i (от 1 до 5). (Знак ≤ используется потому, что один из пловцов не будет назначен ни на один этап).

In [1]:
import pulp
import pandas as pd

# 1. Данные задачи
swimmers = ["Пловец 1", "Пловец 2", "Пловец 3", "Пловец 4", "Пловец 5"]
legs = ["Этап 1", "Этап 2", "Этап 3", "Этап 4"]

# Матрица времени (стоимостей)
times = [
    [37.7, 32.9, 33.8, 37.0, 35.4],  # Этап 1
    [43.4, 33.1, 42.2, 34.7, 41.8],  # Этап 2
    [33.3, 28.5, 38.9, 30.4, 33.6],  # Этап 3
    [29.2, 26.4, 29.6, 28.5, 31.1]   # Этап 4
]

# Создаем словарь стоимостей для удобства
costs = pulp.makeDict([swimmers, legs], [[times[j][i] for j in range(len(legs))] for i in range(len(swimmers))], 0)

# 2. Создание модели
model = pulp.LpProblem("Relay_Team_Optimization", pulp.LpMinimize)

# 3. Создание переменных решения
# assignments[swimmer][leg] = 1, если пловец i назначен на этап j
assignments = pulp.LpVariable.dicts("Assignment", (swimmers, legs), 0, 1, pulp.LpBinary)

# 4. Добавление целевой функции
model += pulp.lpSum([costs[i][j] * assignments[i][j] for i in swimmers for j in legs]), "Total_Relay_Time"

# 5. Добавление ограничений
# Каждый этап должен быть пройден ровно одним пловцом
for j in legs:
    model += pulp.lpSum([assignments[i][j] for i in swimmers]) == 1, f"Leg_{j}_Constraint"

# Каждый пловец может быть назначен не более чем на один этап
for i in swimmers:
    model += pulp.lpSum([assignments[i][j] for j in legs]) <= 1, f"Swimmer_{i}_Constraint"

# 6. Решение модели
model.solve()

# 7. Вывод результатов
print("--- Задание 1: Оптимальная команда и распределение ---")
print("Статус решения:", pulp.LpStatus[model.status])
print(f"Минимальное суммарное время эстафеты: {pulp.value(model.objective):.2f} секунд")

print("\nОптимальный план:")
results = []
for i in swimmers:
    for j in legs:
        if assignments[i][j].varValue == 1:
            results.append({"Этап": j, "Пловец": i, "Время": costs[i][j]})

# Создаем DataFrame для красивого вывода
results_df = pd.DataFrame(results).sort_values(by="Этап")
print(results_df.to_string(index=False))

# Определяем, кто остался в запасе
assigned_swimmers = list(results_df['Пловец'])
reserve_swimmer = [s for s in swimmers if s not in assigned_swimmers][0]
print(f"\nОстается в запасе: {reserve_swimmer}")

--- Задание 1: Оптимальная команда и распределение ---
Статус решения: Optimal
Минимальное суммарное время эстафеты: 126.20 секунд

Оптимальный план:
  Этап   Пловец  Время
Этап 1 Пловец 3   33.8
Этап 2 Пловец 4   34.7
Этап 3 Пловец 2   28.5
Этап 4 Пловец 1   29.2

Остается в запасе: Пловец 5


In [2]:
# --- Задание 2: Обязательное участие 5-го пловца ---

# Создаем новую модель
model_part2 = pulp.LpProblem("Relay_Team_With_Swimmer5", pulp.LpMinimize)

# Переменные и целевая функция те же
assignments_p2 = pulp.LpVariable.dicts("Assignment", (swimmers, legs), 0, 1, pulp.LpBinary)
model_part2 += pulp.lpSum([costs[i][j] * assignments_p2[i][j] for i in swimmers for j in legs]), "Total_Relay_Time_P2"

# Добавляем старые ограничения
for j in legs:
    model_part2 += pulp.lpSum([assignments_p2[i][j] for i in swimmers]) == 1, f"Leg_{j}_Constraint_P2"
for i in swimmers:
    model_part2 += pulp.lpSum([assignments_p2[i][j] for j in legs]) <= 1, f"Swimmer_{i}_Constraint_P2"

# !!! Добавляем НОВОЕ ограничение !!!
# Пловец 5 должен участвовать
model_part2 += pulp.lpSum([assignments_p2["Пловец 5"][j] for j in legs]) == 1, "Swimmer_5_Must_Participate"

# Решаем новую модель
model_part2.solve()

# Вывод результатов
print("\n--- Задание 2: Решение с обязательным участием 5-го пловца ---")
print("Статус решения:", pulp.LpStatus[model_part2.status])
new_time = pulp.value(model_part2.objective)
print(f"Новое минимальное время эстафеты: {new_time:.2f} секунд")

print("\nНовый оптимальный план:")
results_p2 = []
for i in swimmers:
    for j in legs:
        if assignments_p2[i][j].varValue == 1:
            results_p2.append({"Этап": j, "Пловец": i, "Время": costs[i][j]})

results_df_p2 = pd.DataFrame(results_p2).sort_values(by="Этап")
print(results_df_p2.to_string(index=False))

# Определяем, кто остался в запасе
assigned_swimmers_p2 = list(results_df_p2['Пловец'])
reserve_swimmer_p2 = [s for s in swimmers if s not in assigned_swimmers_p2][0]
print(f"\nОстается в запасе: {reserve_swimmer_p2}")

# Сравниваем результаты
time_increase = new_time - pulp.value(model.objective)
print("\n--- Анализ изменений ---")
print(f"Суммарное время увеличилось на: {time_increase:.2f} секунд ({new_time:.2f} - {pulp.value(model.objective):.2f})")
print("Это произошло потому, что принудительное включение 5-го пловца в команду заставило отказаться от более оптимального состава, где участвовал Пловец 1, а Пловец 3 был в запасе.")


--- Задание 2: Решение с обязательным участием 5-го пловца ---
Статус решения: Optimal
Новое минимальное время эстафеты: 127.80 секунд

Новый оптимальный план:
  Этап   Пловец  Время
Этап 1 Пловец 5   35.4
Этап 2 Пловец 4   34.7
Этап 3 Пловец 2   28.5
Этап 4 Пловец 1   29.2

Остается в запасе: Пловец 3

--- Анализ изменений ---
Суммарное время увеличилось на: 1.60 секунд (127.80 - 126.20)
Это произошло потому, что принудительное включение 5-го пловца в команду заставило отказаться от более оптимального состава, где участвовал Пловец 1, а Пловец 3 был в запасе.


1. Привести математическую модель классической транспортной задачи. Объяснить смысл переменных, целевой функции и ограничений.
Переменные (xᵢⱼ): Количество груза, перевозимого со склада i потребителю j.
Целевая функция: Минимизация общих затрат на перевозку. Рассчитывается как СУММА (Стоимостьᵢⱼ * xᵢⱼ) по всем маршрутам.
Ограничения:
По запасам: С каждого склада i нельзя вывезти больше груза, чем там есть.
По потребностям: Каждому потребителю j нужно доставить ровно столько груза, сколько он заказывает.
2. Какие имеются формы представления математической модели транспортной задачи?
Матричная (табличная): Самая наглядная. В таблице указаны поставщики (строки), потребители (столбцы), а в ячейках — стоимость перевозки. Запасы и потребности указаны по краям таблицы.
Сетевая: В виде графа, где узлы — это поставщики и потребители, а дуги (стрелки) — это маршруты перевозок.
Модель линейного программирования: Формальная математическая запись с переменными, целевой функцией и системой ограничений (см. вопрос 1).
3. Дать определение сбалансированной транспортной задачи. Почему это важно?
Определение: Задача сбалансирована, если суммарный запас всех поставщиков равен суммарной потребности всех потребителей.
Почему важно: Классические методы решения (например, метод потенциалов) разработаны именно для сбалансированных задач. Если задача не сбалансирована, ее нужно сначала искусственно "сбалансировать", чтобы метод сработал.
4. Следует ли вводить условие целочисленности переменных при решении сбалансированной транспортной задачи? Объяснить почему.
Нет, не следует. У сбалансированной транспортной задачи есть уникальное свойство: если все запасы и потребности — целые числа, то ее оптимальное решение, найденное симплекс-методом, автоматически будет целочисленным. Вводить дополнительное условие целочисленности не нужно, это только усложнит вычисления.
5. Как привести задачу к сбалансированному виду, если запасы превышают заказы? Как определить, сколько останется у каждого поставщика?
Как привести: Вводится фиктивный потребитель, потребность которого равна общему излишку (Сумма запасов - Сумма потребностей).
Как определить остаток: Количество товара, "отправленное" от реального поставщика фиктивному потребителю в оптимальном плане, — это и есть остаток товара на складе этого поставщика.
6. Как привести задачу к сбалансированному виду, если заказы превышают запасы? Как определить, кто сколько недополучит?
Как привести: Вводится фиктивный поставщик, запас которого равен общему дефициту (Сумма потребностей - Сумма запасов).
Как определить недопоставку: Количество товара, "полученное" реальным потребителем от фиктивного поставщика, — это и есть объем недопоставки для этого потребителя.
7. Каким образом следует модифицировать модель при наличии дополнительных ограничений?
а) Доставка из Sᵢ в Dⱼ запрещена: Просто присвоить стоимости перевозки по этому маршруту очень большое число ("бесконечность" или "штраф М"). Решатель никогда не выберет такой дорогой путь.
б) Доставить ровно N единиц: Добавить ограничение-равенство: xᵢⱼ = N.
в) Доставить не менее N единиц: Добавить ограничение-неравенство: xᵢⱼ ≥ N.
г) Доставить не более N единиц: Добавить ограничение-неравенство: xᵢⱼ ≤ N.
8. Привести математическую модель задачи о назначениях. Объяснить смысл.
Переменные (xᵢⱼ): Бинарные (0 или 1). xᵢⱼ = 1, если исполнитель i назначен на работу j, и 0 — если нет.
Целевая функция: Минимизация суммарных затрат (времени, стоимости) на выполнение всех работ. СУММА (Затратыᵢⱼ * xᵢⱼ).
Ограничения:
Каждый исполнитель i назначается ровно на одну работу.
Каждая работа j выполняется ровно одним исполнителем.
9. Что общего и в чем различия между транспортной задачей и задачей о назначениях?
Общее: Обе задачи относятся к распределительному типу и могут быть решены методами линейного программирования. Задача о назначениях — это частный случай транспортной задачи.
Различия:
Тип переменных: В ТЗ — непрерывные (сколько везти), в ЗН — бинарные (назначен/не назначен).
Запасы/Потребности: В ЗН все "запасы" (исполнители) и "потребности" (работы) равны 1.
Смысл: ТЗ — "сколько откуда куда везти", ЗН — "кого на какую работу назначить".
10. В чем проявляется несбалансированность задачи о назначениях? Как ее сбалансировать?
Проявление: Количество исполнителей не равно количеству работ.
Как сбалансировать:
Недостаток исполнителей: Добавляются фиктивные исполнители, которые могут выполнить любую работу с нулевой (или штрафной) стоимостью. Работы, назначенные им, останутся невыполненными.
Избыток исполнителей: Добавляются фиктивные работы. Исполнители, назначенные на них, останутся без работы.
11. Привести содержательную постановку транспортной задачи с промежуточными пунктами.
Это задача, где груз едет не напрямую от поставщика к потребителю, а может проходить через транзитные узлы (склады, порты, хабы).
Отличие от классической ТЗ: Появляется третий тип узлов — промежуточные, которые могут и получать, и отправлять груз. Главное условие для них — "сколько вошло, столько и вышло" (закон сохранения потока).
12. Привести формальную модель узла сети для ТЗ с промежуточными пунктами. В чем отличия?
Источник (поставщик): Сумма исходящих потоков ≤ Запаса.
Сток (потребитель): Сумма входящих потоков = Потребности.
Промежуточный узел (транзит): Сумма входящих потоков = Сумме исходящих потоков. Это ключевое отличие.
13. Условие сбалансированности ТЗ с промежуточными пунктами. Как ее сбалансировать?
Условие: Сумма запасов всех источников равна сумме потребностей всех стоков. Промежуточные пункты на баланс не влияют.
Сбалансировать: Так же, как и в классической ТЗ — введением фиктивного поставщика (если дефицит) или фиктивного потребителя (если избыток).
14. Привести и пояснить схему приведения ТЗ с промежуточными пунктами к классической.
Это делается созданием "большой" матрицы классической ТЗ:
Источниками в новой матрице будут реальные источники И промежуточные пункты.
Стоками в новой матрице будут реальные стоки И промежуточные пункты.
Запас промежуточного пункта берется очень большим (буферный запас), чтобы через него могло пройти сколько угодно груза.
Перевозка из промежуточного пункта в него же (по диагонали) делается бесплатной (стоимость 0).
15. Для чего вводятся переменные xₖₖ? Какова их интерпретация и целевые коэффициенты?
Для чего: Это переменные, отвечающие за перевозку из промежуточного пункта k в него же.
Интерпретация: xₖₖ — это количество транзитного груза, который проходит через узел k.
Целевые коэффициенты: Их стоимость (коэффициент cₖₖ) всегда равна нулю, так как это не реальная перевозка, а математический трюк для моделирования транзита.
16. Каким образом задача поиска кратчайшего пути может быть представлена как ТЗ с промежуточными пунктами?
Начальный пункт: Источник с запасом = 1.
Конечный пункт: Сток с потребностью = 1.
Все остальные пункты: Промежуточные узлы.
Стоимость перевозки: Длина (или стоимость) пути между узлами.
Решение: Оптимальный план покажет маршрут с единичным потоком от начала до конца, и его стоимость будет равна длине кратчайшего пути.
17. Привести постановку задачи об аренде оборудования. Как ее представить в виде ТЗ?
Постановка: Нужно определить, в какие моменты времени арендовать новое оборудование и на какой срок, чтобы минимизировать суммарные затраты на аренду и обслуживание за весь период планирования.
Представление в виде ТЗ: Это задача о кратчайшем пути на специальном графе.
Узлы: Моменты времени (начало 1-го года, 2-го, ..., конца 5-го).
Дуги (маршруты): Аренда оборудования в момент i и его использование до момента j.
Стоимость дуги: Стоимость аренды и обслуживания на отрезке времени от i до j.
Задача: Найти кратчайший (самый дешевый) путь из начального узла (старт 1-го года) в конечный (конец 5-го года).